In [1]:
# define matplotlibplotting backend
%matplotlib qt

In [2]:
from lib.parameters import Parameters
from lib.plot_builder import PlotBuilder
from lib.h5_reader import H5Reader
from lib.plain_reader import PlainReader

/home/cosmonaut/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
##  configuration options
#  set color limit for E_r, E_z and rho_beam
clim_e_r = clim_e_z = [-5e5, 5e5]
clim_rho_beam = [0, 1e15]

#  set timestamp
timestamp = 1e-9

# set color map
cmap = 'terrain'

# define configuration file
config_file = '/home/cosmonaut/pdp3_modeling/model32_single/parameters.xml'

In [4]:
## read configfile
cfg = Parameters(config_file)

# define reader (plain reader used)
reader = PlainReader(cfg.data_path, cfg.system_state_path,
                     [cfg.number_r_grid , cfg.number_z_grid],
                     cfg.frames_per_file, False)

# define plot builder
plot = PlotBuilder(cfg.number_z_grid, cfg.number_r_grid,
                   fig_color=cfg.figure_color, fig_width=cfg.figure_width,
                   fig_height=cfg.figure_height, fig_dpi=cfg.figure_dpi,
                   font_family=cfg.figure_font_family,
                   font_name=cfg.figure_font_name,
                   font_size=cfg.figure_font_size,

                   x_ticklabel_end=cfg.z_size, y_ticklabel_end=cfg.r_size,
                   tickbox=True, grid=True, is_invert_y_axe=False,
                   aspect='equal', image_interpolation='nearest')

In [5]:
# add subplots
plot.add_subplot_cartesian_2d(r'$E_r$', 111)

In [7]:
cfg.r_size

0.15

In [6]:
frame_number = cfg.get_frame_number_by_timestamp(timestamp)
data = reader.get_frame('E_r', frame_number)

In [7]:
# read and add images to subplots
plot.add_image(r'$E_r$', data, cmap=cmap, clim=clim_e_r)

AttributeError: 'PlotBuilder' object has no attribute 'default_image_interpolation'

In [173]:
plot.show()

In [21]:
rc

NameError: name 'rc' is not defined

In [22]:
from matplotlib import rc

In [23]:
rc

<function matplotlib.rc(group, **kwargs)>

In [146]:
x_plot_size = 0.002
y_plot_size = 0.00015
z_plot_size = 0.031458

In [147]:
max_size = np.max([x_plot_size, y_plot_size, z_plot_size])

In [163]:
exp_number = np.floor(np.log10(max_size))

In [164]:
exp_number

-2.0

In [10]:
import numpy as np

In [11]:
def basenumbercalc(a, b, c):
    max_size = np.max([a, b, c])
    exp_number = np.floor(np.log10(max_size))
    base_number = int(max_size / pow(10, exp_number))
    unit = (base_number / 40)
    unit = unit * pow(10, exp_number)
    return unit

In [12]:
basenumbercalc(1.2, 0.15, 0)

0.025

In [14]:
int(1.2 / 0.025)

47

In [20]:
np.around(np.linspace(0, 1.2, 47), 2)

array([0.  , 0.03, 0.05, 0.08, 0.1 , 0.13, 0.16, 0.18, 0.21, 0.23, 0.26,
       0.29, 0.31, 0.34, 0.37, 0.39, 0.42, 0.44, 0.47, 0.5 , 0.52, 0.55,
       0.57, 0.6 , 0.63, 0.65, 0.68, 0.7 , 0.73, 0.76, 0.78, 0.81, 0.83,
       0.86, 0.89, 0.91, 0.94, 0.97, 0.99, 1.02, 1.04, 1.07, 1.1 , 1.12,
       1.15, 1.17, 1.2 ])

In [37]:
np.linspace(0, 0.23,0.005)

/home/cosmonaut/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  """Entry point for launching an IPython kernel.


array([], dtype=float64)

In [38]:
(0.23 - 0) / 0.005

46.0

In [65]:
0.23 / 0.0005

460.0